# MRI Fuzzy Segmentation Region Growing

# Introduction to Stroke Lesion Segmentation Project

Hello! My name is Mario Pascual Gonzalez, and I am a Bioinformatics student at the University of Málaga. This Jupyter notebook contains all the code that I've developed for my final project in the course "Biomedical Imaging." 

Under the guidance of Dr. Enrique Nava Baro, this project aims to address the significant challenge of segmenting stroke lesions from brain MRI images. The focus is on applying the Fuzzy Information Seeded Region Growing (FISRG) algorithm to enhance the accuracy and efficiency of this crucial task in medical imaging.

Within these notebooks, you will find detailed code, analysis, and insights that collectively form the backbone of my research efforts in this area. Each section is carefully documented to provide clarity on the methodologies and processes involved.

Thank you for your interest in my work, and I hope it provides valuable insights and contributes meaningfully to the field of biomedical imaging.



In [58]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import os
%matplotlib notebook
from sklearn.cluster import KMeans

## Image visualization

In [56]:
def overlay_mask_on_image(image, mask, color=(255, 0, 0)):
    """
    Applies a colored mask over the original grayscale image to highlight the regions of interest.

    @:param mask: A binary mask where non-zero (typically 255) indicates the regions of interest.
    @:type: numpy.ndarray
    @:param color: A 3-tuple representing the BGR color (Blue, Green, Red) to overlay where the mask is
    active. Default is red.
    @:type: tuple

    @:return color_image (numpy.ndarray): The original image with the mask overlay in the specified color.
    """
    # Create a 3-channel version of the grayscale image
    color_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    # Identify where the mask is active
    active_pixels = (mask == 255)

    # Apply color to the active mask regions in the image
    color_image[active_pixels] = color

    return color_image


def visualize(image, truth, prediction):
    """
    Displays the original image, the mask, and the overlay of the mask on the original image side by side for
    comparison.

    @:param mask: A binary mask where non-zero (typically 255) indicates the regions of interest.
    @:type numpy.ndarray
    @:param color: A 3-tuple representing the BGR color (Blue, Green, Red) to overlay on the mask.
    @:type tuple
    Default is red.
    """
    _, axes = plt.subplots(nrows=1, ncols=3, figsize=(10,6))

    axes[0].imshow(image, cmap='gray')
    axes[0].set_title('Original Image')
    axes[0].axis('off')

    axes[1].imshow(prediction, cmap='gray')
    axes[1].set_title('Mask')
    axes[1].axis('off')

    transparent_overlaying(axes[2], truth, prediction, image)

def transparent_overlaying(ax, ground_truth, pred_mask, mri):
    """
    Overlay ground truth and prediction masks on an MRI image for visualization.

    This function displays an MRI image and overlays it with two masks: the ground truth and the predicted mask.
    The ground truth mask is shown in green, and the predicted mask is shown in red, both with a degree of transparency.
    This visualization aids in comparing the predicted results against the actual ground truth.

    :param ax: matplotlib.axes.Axes, the axes object where the image is to be displayed
    :param ground_truth: numpy.ndarray, the ground truth mask for the MRI image
    :param pred_mask: numpy.ndarray, the predicted mask for the MRI image
    :param mri: numpy.ndarray, the actual MRI image

    :return: None
    """
    import matplotlib.colors as mcolors
    
    # Display the MRI image in grayscale
    ax.imshow(mri, cmap='gray')
    
    # Define custom color maps for ground truth and prediction masks
    cmap_ground_truth = mcolors.ListedColormap(['none', 'green'])
    cmap_prediction = mcolors.ListedColormap(['none', 'red'])
    
    # Overlay the ground truth mask with green color and transparency
    ax.imshow(ground_truth, cmap=cmap_ground_truth, alpha=0.3)
    
    # Overlay the prediction mask with red color and transparency
    ax.imshow(pred_mask, cmap=cmap_prediction, alpha=0.3)
    
    # Remove axes if not needed
    ax.axis('off')
    
    # Set title and create a legend
    ax.set_title(f'Predictions')
    # Define legend elements for ground truth and prediction
    legend_elements = [plt.Line2D([0], [0], marker='o', color='w', label='Ground Truth',
                                  markersize=10, markerfacecolor='green', alpha=0.5),
                       plt.Line2D([0], [0], marker='o', color='w', label='Prediction',
                                  markersize=10, markerfacecolor='red', alpha=0.5)]
    # Add the legend to the plot
    ax.legend(handles=legend_elements, loc='upper right')

## Dice Coefficient for evaluation

In [32]:
def dice_coefficient(mask1, mask2):
    """
    Calculate the Dice Coefficient between two binary masks.
    
    Parameters:
    - mask1 (numpy.ndarray): 2D binary array, where 1's represent the first segmented region.
    - mask2 (numpy.ndarray): 2D binary array, where 1's represent the second segmented region.
    
    Returns:
    - dice_coeff (float): Dice Coefficient as a float between 0.0 and 1.0.
    """
    mask1 = np.where(mask1 == 255, 1, 0)
    mask2 = np.where(mask2 == 255, 1, 0)
    # Calculate intersection and union
    intersection = np.logical_and(mask1, mask2).sum()
    union = mask1.sum() + mask2.sum()

    # Calculate Dice Coefficient
    dice_coeff = 2. * intersection / union if union != 0 else 1.0

    return dice_coeff

## Load NiFTI file

In [4]:
path2 = "C://Users\mario\Python\Projects\MRIfuzzySegmentation\Sample data"
mri_image = nib.load(os.path.join(path2, "sub-r001s001_ses-1_space-MNI152NLin2009aSym_T1w.nii.gz")).get_fdata()
mri_mask = nib.load(os.path.join(path2, "sub-r001s001_ses-1_space-MNI152NLin2009aSym_label-L_desc-T1lesion_mask.nii.gz")).get_fdata()


**Loading MRI Image and Mask Data**

The code snippet is responsible for loading MRI image data and its corresponding mask from a specified directory. It utilizes the `nibabel` library to handle `.nii.gz` files, which are commonly used formats for storing MRI data.

Firstly, the path to the directory containing the sample data is defined. Then, the MRI image (`sub-r001s001_ses-1_space-MNI152NLin2009aSym_T1w.nii.gz`) and its associated mask (`sub-r001s001_ses-1_space-MNI152NLin2009aSym_label-L_desc-T1lesion_mask.nii.gz`) are loaded from this directory. The `get_fdata()` method is called on each loaded file to extract the functional data as NumPy arrays. This process facilitates subsequent processing and analysis of the MRI image and its mask in the neuroimaging workflow.



## Manually set seeds for the FISRG Algorithm -Code used to research seed impact on segmentation on early stages-

In [5]:
# Function to handle mouse events
def on_mouse(event, x, y, flags, param):
    image, seeds = param
    if event == cv2.EVENT_LBUTTONDOWN:
        # print(f'Seed: {x}, {y}, Pixel value: {image[y, x]}')
        seeds.append((y, x))

def set_seeds(image):
    # Initialize an empty list to store the seed points
    seeds = []
    # Create an OpenCV window
    cv2.namedWindow('TC Image')
    # Set the mouse callback function
    cv2.setMouseCallback('TC Image', on_mouse, [image, seeds])
    while True:
        # Show the image in the OpenCV window
        cv2.imshow('TC Image', image)
        # Wait for a key event
        key = cv2.waitKey(1) & 0xFF
        # Break the loop if the 'q' key is pressed
        if key == ord('q'):
            break
    # Destroy all OpenCV windows
    cv2.destroyAllWindows()
    return seeds

**Implementing Interactive Seed Selection**

In this part of my project, I've implemented two functions, `on_mouse` and `set_seeds`, to facilitate interactive seed selection directly on an image using OpenCV's graphical user interface.

1. **`on_mouse` Function**: This function is crucial for handling mouse events. I designed it to activate whenever there's a mouse event in the OpenCV window. Specifically, when I click the left mouse button, it captures the x and y coordinates of that click point. These coordinates are critical as they represent a seed point on the image. I append each of these seed points to a list, effectively collecting all the seeds I need for further processing.

2. **`set_seeds` Function**: This function kicks off the interactive seed selection process. It takes an image as its input and then opens an OpenCV window to display this image. I've set the `on_mouse` function as the callback for handling mouse events in this window. The key part here is the continuous display of the image in the OpenCV window, which I achieve using `cv2.imshow`. This loop continues until I press the 'q' key, signaling the end of the seed selection process. Once I press 'q', the window closes, and the function returns all the seeds I've selected.


## Automatically set seeds

In [6]:
def is_valid_seed(image, seed, max_std, existing_seeds, min_dist):
    """
    Check if a seed is valid based on the homogeneity of its 8 neighbors, standard deviation,
    and minimum distance from existing seeds.
    
    :param image: The input image where the seed lies.
    :param seed: The (x, y) coordinates of the seed.
    :param max_std: The maximum allowed standard deviation for the neighbors.
    :param existing_seeds: List of already validated seeds to check the distance against.
    :param min_dist: The minimum distance required between seeds.
    :return: Boolean value indicating whether the seed is valid.
    """
    x, y = seed
    neighbors = image[max(0, x-1):x+2, max(0, y-1):y+2]
    if neighbors.size < 9 or np.std(neighbors) > max_std:
        return False

    for ex_seed in existing_seeds:
        if np.sqrt((ex_seed[0] - x) ** 2 + (ex_seed[1] - y) ** 2) < min_dist:
            return False

    return True

def automatic_seed_distribution(image, mask, n_seeds, max_std, min_dist):
    """
    Distribute a specified number of seeds across the largest contiguous region in a binary mask using KMeans clustering.
    The function identifies the largest contour in the mask and attempts to distribute the seeds within it, ensuring that each seed meets the specified standard deviation and minimum distance criteria.

    :param n_seeds: int, the number of seeds to be distributed within the largest contour
    :param max_std: float, the maximum allowable standard deviation for seed validation
    :param min_dist: float, the minimum distance required between seeds for validation

    :return: tuple(list, numpy.ndarray)
        - list of tuples, where each tuple represents the coordinates of a valid seed
        - numpy.ndarray representing the largest contour found in the binary mask

    This function first converts the mask to uint8, finds contours, and identifies the largest contour. It then creates a new ROI mask and extracts points within this ROI for clustering. KMeans clustering is applied to distribute seeds, which are then validated based on the specified criteria. If the desired number of valid seeds is not achieved, the process is repeated with a reduced number of seeds.
    """
    maskuint8 = mask.astype(np.uint8)
    # Encontrar contornos en una copia de la máscara para evitar modificar la original
    contours, _ = cv2.findContours(maskuint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2:]    
    # Encuentra el contorno más grande (suponiendo que es el ROI)
    largest_contour = max(contours, key=cv2.contourArea)
    
    # Crea una nueva máscara para el ROI
    roi_mask = np.zeros((maskuint8.shape[0], maskuint8.shape[1], 3), dtype=np.uint8)
    cv2.drawContours(roi_mask, [largest_contour], -1, (255, 255, 255), thickness=-1)
    
    # Obtener puntos dentro del ROI para clústering
    roi_mask = roi_mask[:, :, 0]
    X, Y = np.nonzero(roi_mask)
    points = np.column_stack((X, Y))  
    valid_seeds = []
    attempts = 0
    max_attempts = 100
    
    while len(valid_seeds) < n_seeds and attempts < max_attempts:
        attempts += 1
        kmeans = KMeans(n_clusters=n_seeds, n_init=10, random_state=42).fit(points)
        centroids = kmeans.cluster_centers_

        for center in centroids:
            pt = (int(center[0]), int(center[1]))
            if cv2.pointPolygonTest(largest_contour, (pt[1], pt[0]), False) >= 0:
                if is_valid_seed(image, pt, max_std, valid_seeds, min_dist):
                    valid_seeds.append(pt)

        if len(valid_seeds) < n_seeds:
            valid_seeds.clear()  # Clear the list and try again
            n_seeds -= 1  # Reduce the number of seeds for the next iteration
            
    return [tuple(seed) for seed in valid_seeds]


In my image processing pipeline, I have developed two functions, `is_valid_seed` and `automatic_seed_distribution`, which are central to the seed-based segmentation approach.

**Function: `is_valid_seed`**
This function is designed to validate a seed point in an image. A seed is considered valid if it meets two conditions: homogeneity in its immediate neighborhood and a minimum distance from previously selected seeds. 

- The function checks the standard deviation of the 8 neighboring pixels around the seed. If this standard deviation is higher than a specified threshold (`max_std`), the seed is deemed invalid, as it indicates a high level of heterogeneity.
- It also ensures that each new seed is at a minimum distance (`min_dist`) from already existing seeds. This prevents clustering of seeds and promotes a more uniform seed distribution.

**Function: `automatic_seed_distribution`**
This function automates the seed distribution process across the largest contiguous region in a binary mask. 

- It starts by identifying the largest contour within a given mask, which is assumed to be the region of interest (ROI).
- Utilizing KMeans clustering, it attempts to distribute a specified number of seeds (`n_seeds`) within this ROI. 
- Each potential seed generated by KMeans is then passed to the `is_valid_seed` function to ensure it meets the criteria for homogeneity and distance.
- The process iteratively adjusts the number of seeds and repeats until the desired number of valid seeds is achieved or the maximum number of attempts is reached.

These functions are integral to ensuring accurate and efficient seed placement in the segmentation process, particularly for our images where precise localization is critical.


## Fuzzy Information Seeded Region Growing (FISRG) Algorithm

### Fuzzy predicate

In [7]:
def gaussian_pdf(x, mean, std):
    """
    Calculate the Gaussian probability density function for a given x with specified mean and standard deviation.

    :param x: The value (intensity of the pixel) for which the PDF is being calculated.
    :param mean: The mean value of the Gaussian distribution (mean intensity of the tumor region).
    :param std: The standard deviation of the Gaussian distribution (standard deviation of intensity in the tumor region).
    :return: The probability density value for the given x.
    """
    epsilon = 1e-8  # You can adjust this threshold as necessary.
    std = max(std, epsilon)
    return (1.0 / (np.sqrt(2 * np.pi * std**2))) * np.exp(-((x - mean)**2 / (2 * std**2)))

In my fuzzy image segmentation research, particularly for the Fuzzy Region Growing (FISRG) algorithm, a critical component is the fuzzy predicate. This predicate is defined by the Gaussian Probability Density Function (PDF), which I implement in the `gaussian_pdf` function.

**Function: `gaussian_pdf`**
This function calculates the Gaussian PDF for a given pixel intensity value (`x`). The Gaussian distribution is characterized by two parameters: the mean (`mean`) and the standard deviation (`std`). These parameters are central to defining the shape of the Gaussian curve, which in turn influences the segmentation process. 

- The mean typically represents the average intensity of the tumor region in medical imaging. 
- The standard deviation provides a measure of the spread or dispersion of pixel intensities around this mean.

The function includes a safeguard against division by zero or extremely small standard deviation values by introducing a small value, `epsilon`. This ensures numerical stability during computation.

The Gaussian PDF plays a crucial role in the FISRG algorithm. It determines the likelihood that a given pixel belongs to a region of interest, such as a tumor, based on its intensity value. A higher probability density value indicates a closer resemblance to the tumor's characteristics, guiding the region growing process in segmenting the image more accurately.

$$
    S(x) = \frac{1}{\sqrt{2\pi\sigma^2}} e^{-\frac{(x - \mu)^2}{2\sigma^2}}

$$


### Get 8 surrounding neightbours

In [8]:
def get_8_connected(x, y, shape):
    """
    Generates a list of 8-connected neighbors for a given pixel within the specified shape.

    :param x: The x-coordinate (column) of the current pixel.
    :param y: The y-coordinate (row) of the current pixel.
    :param shape: The shape (dimensions) of the image array with format (rows, columns).
    :return: A list of tuples representing the 8-connected neighboring pixel coordinates.
    """
    rows, cols = shape  # The correct order is rows, cols, not xmax, ymax.
    connected_pixels = []

    # Check neighbors with correct bounds checking
    for dx in range(-1, 2):  # -1, 0, +1
        for dy in range(-1, 2):  # -1, 0, +1
            # Calculate neighbor coordinates
            connected_pixel_x = x + dx
            connected_pixel_y = y + dy
            # Skip out-of-bounds coordinates or the same center point
            if connected_pixel_x < 0 or connected_pixel_x >= cols or \
               connected_pixel_y < 0 or connected_pixel_y >= rows or \
               (dx == 0 and dy == 0):
                continue  # Skip iteration
            connected_pixels.append((connected_pixel_x, connected_pixel_y))

    return connected_pixels

In the development of the Fuzzy Region Growing (FISRG) algorithm for image segmentation, I've implemented a function named `get_8_connected`. This function plays a crucial role in identifying the neighborhood of a given pixel, which is a key step in the region-growing process.

**Function: `get_8_connected`**
The purpose of this function is to generate a list of 8-connected neighbors for a specific pixel. This connectivity is essential for examining the surrounding pixels and determining whether they should be included in the same segment as the central pixel.

- It takes the coordinates (`x`, `y`) of a pixel and the shape of the image as parameters. The shape is expected to be in the format of (rows, columns).
- The function iteratively checks all potential neighboring pixels, considering both horizontal, vertical, and diagonal connections.
- An important aspect of the function is boundary checking. It ensures that the identified neighbors are within the image bounds and excludes the central pixel itself from the list of neighbors.

This 8-connected neighborhood approach is aligned with the concept of fuzzy connectedness used in the FISRG algorithm. It allows for a more comprehensive and contiguous region growth, as opposed to a 4-connected approach which might miss certain diagonally connected pixels. 



### Algorithm for one seed

In [5]:
def grow_region(image, seed_point, threshold):
    """
    Perform region growing in an image from a given seed point based on pixel intensity similarity.
    
    This function segments an image by starting from a seed point and adding neighboring pixels to the
    region if their intensity is similar to the region's current intensity statistics. The similarity
    is determined using a fuzzy similarity measure based on Gaussian probability density function (PDF).
    
    Parameters:
    - image (numpy.ndarray): The input image where the region is to be grown.
    - seed_point (tuple): A tuple (x, y) representing the seed point's coordinates.
    - threshold (float): A threshold value for the fuzzy similarity. Pixels with similarity scores above
                         this threshold will be added to the region.
    
    Returns:
    - region_mask (numpy.ndarray): A binary mask of the same size as 'image' with the grown region marked.
    
    The algorithm maintains a list of points 'to_process' that need to be examined. A point is only
    processed if it has not been processed before. The 'processed' set keeps track of all points that
    have already been evaluated. For each point, the function calculates its similarity to the seed point's
    intensity and includes it in the region if the similarity is above the specified threshold. The region's
    mean and standard deviation are updated dynamically as new pixels are added.
    
    The algorithm iterates until there are no more points to process, ensuring that all potential pixels
    are evaluated for inclusion in the region. The result is a binary mask representing the segmented region.
    """

    # Get the dimensions of the input image
    height, width = image.shape[:2]

    # Initialize a binary mask to store the segmented region
    region_mask = np.zeros((height, width), np.uint8)

    # Initialize a list with the seed point
    to_process = [seed_point]

    # Initialize a set to keep track of processed points
    processed = set()

    # Initialize variables to compute the mean and standard deviation of the region's intensity
    total_intensity = 0
    total_squared_intensity = 0
    num_pixels = 0

    # Initialize with the seed point's intensity
    x, y = seed_point
    seed_intensity = image[y, x]
    mean_val = seed_intensity
    std_val = 0  # Start with zero deviation as there's only one pixel in the region initially

    # Loop through all points to be processed
    while to_process:
        # Pop the first point from the list
        point = to_process.pop(0)

        # Skip if the point has already been processed
        if point in processed:
            continue

        # Mark the point as processed
        processed.add(point)

        # Extract the point's coordinates and pixel intensity
        x, y = point
        pixel_intensity = float(image[y, x])

        # Update the total intensity and number of pixels to compute the new mean and standard deviation
        total_intensity += pixel_intensity
        total_squared_intensity += pixel_intensity**2
        num_pixels += 1

        # Calculate the new mean and standard deviation of the region
        mean_val = total_intensity / num_pixels
        std_val = np.sqrt((total_squared_intensity / num_pixels) - (mean_val**2))

        # Compute the fuzzy similarity for the current pixel
        similarity = gaussian_pdf(pixel_intensity, mean_val, std_val)

        # If the similarity is above the threshold, include the pixel in the region
        if similarity >= threshold:
            region_mask[y, x] = 255  # Mark the pixel in the region mask

            # Get the 8-connected neighbors of the current point
            neighbors = get_8_connected(x, y, image.shape)

            # Add neighbors to the 'to_process' list if they haven't been processed yet
            for neighbor in neighbors:
                if neighbor not in processed:
                    to_process.append(neighbor)

    return region_mask

In my work on image segmentation using the Fuzzy Region Growing (FISRG) algorithm, a key function I implemented is `grow_region`. This function is fundamental for segmenting a region in an image based on fuzzy similarity, starting from a specified seed point.

**Function: `grow_region`**
The primary objective of this function is to segment an area of the image that is similar in characteristics to the seed point. This process is based on the fuzzy similarity concept, which evaluates how close the neighboring pixels are to the seed point in terms of intensity.

- The function initializes by creating a mask to represent the segmented region and setting up lists for pixels to be processed and those already processed.
- It begins with the provided seed point, calculating the mean and standard deviation of the pixel intensities in the growing region. Initially, these are based solely on the seed point's intensity.
- As the region grows, the mean and standard deviation are dynamically updated to reflect the intensities of all pixels included in the region.
- For each pixel being processed, I calculate its fuzzy similarity to the region using the `gaussian_pdf` function. If this similarity exceeds a predefined threshold, the pixel is added to the region.
- The function ensures connectivity by considering 8-connected neighbors of each pixel and includes them in the region if they meet the similarity criterion.

This approach allows for adaptively growing the region based on the local properties of the image, leading to more accurate and coherent segmentation, especially in medical images where precise identification of regions is crucial.


### Full algorithm call for various seeds

In [10]:
def region_grow_all_seeds(image, seed_points, acceptance_probability):
    """
    Segment regions in an image using multiple seeds and return the logical OR of all masks calculated.

    :param image: The MRI image to be segmented.
    :param seed_points: A list of tuples, where each tuple represents a seed point.
    :param acceptance_probability: The probability threshold for accepting pixels into the region.
    :return: A binary mask representing the union of all regions grown from the seed points.
    """
    combined_mask = np.zeros(image.shape[:2], dtype=bool)  # Initialize a mask to store the combined region.

    for seed_point in seed_points:
        # Call the region grow function for each seed
        region_mask = grow_region(image, seed_point, acceptance_probability)
        # Combine the current region mask with the combined mask using logical OR
        combined_mask = np.logical_or(combined_mask, region_mask.astype(bool))

    # If necessary, convert the boolean mask to a format that can be used/displayed by other libraries (e.g., multiplied by 255 for OpenCV)
    return combined_mask.astype(np.uint8) * 255

In my Fuzzy Region Growing (FISRG) algorithm for image segmentation, I've implemented the `region_grow_all_seeds` function. This function is instrumental in segmenting multiple regions from a set of given seed points and combining their results into a single comprehensive mask.

**Function: `region_grow_all_seeds`**
The function's main goal is to apply the region growing process to each seed point in a list and then amalgamate the resulting segmented regions. 

- The process starts by initializing a combined mask, which will eventually represent the union of all segmented regions.
- For each seed point in the `seed_points` list, I invoke the previously defined `grow_region` function. This function segments the region around the seed point based on the given `acceptance_probability` threshold, which is the fuzzy similarity criterion.
- After obtaining the region mask for each seed, I merge it with the combined mask using a logical OR operation. This step ensures that all the segmented regions are included in the final output.
- Finally, I return the combined mask as a binary image, which can be easily integrated with other image processing libraries or displayed for analysis.

This approach allows for a more comprehensive segmentation when dealing with images that have multiple areas of interest, such as in complex medical imaging scenarios where multiple regions need to be analyzed simultaneously.


### Example of usage

In [60]:
thisMRI = mri_image[:, :, 77].astype(np.uint8)
thisMask = mri_mask[:, :, 77].astype(np.uint8) * 255

number_of_seeds = 4
seeds, largest_c = automatic_seed_distribution(thisMRI, thisMask, number_of_seeds, 20, 2)
seeds = [(y, x) for x, y in seeds]

# Preprocess the image using a Gaussian blur 
this_final = cv2.GaussianBlur(thisMRI, (0,0), 2)
# ===============================================

fuzzy_threshold = .112

region_mask = region_grow_all_seeds(this_final, seeds, fuzzy_threshold)

_, axes = plt.subplots()
transparent_overlaying(axes, thisMask, region_mask, thisMRI)
plt.show()

C:\Users\mario\miniconda3\envs\imbio\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=7.
  warnings.warn(


<IPython.core.display.Javascript object>

## Post-Processing: Mathematical Morphology

In [40]:
def improve_mask(mask, operation='open', kernel_size=3, iterations=1):
    """
    Perform morphological operations to improve the properties of a binary mask.

    Parameters:
    - mask (numpy.ndarray): 2D binary mask to be improved.
    - operation (str): Morphological operation to be performed ('open', 'close', 'erode', 'dilate').
    - kernel_size (int): Size of the kernel used for the morphological operation.
    - iterations (int): Number of times the operation is applied.

    Returns:
    - improved_mask (numpy.ndarray): The improved binary mask.
    """

    # Create the kernel for the morphological operation
    kernel = np.ones((kernel_size, kernel_size), np.uint8)

    # Dictionary of available morphological operations
    operations = {
        'open': cv2.MORPH_OPEN,      # Opening (erosion followed by dilation)
        'close': cv2.MORPH_CLOSE,    # Closing (dilation followed by erosion)
        'erode': cv2.MORPH_ERODE,    # Erosion
        'dilate': cv2.MORPH_DILATE,  # Dilation
    }

    if operation not in operations:
        raise ValueError(f"Operation '{operation}' is not supported. Choose from 'open', 'close', 'erode', 'dilate'.")

    # Perform the chosen morphological operation
    improved_mask = cv2.morphologyEx(mask, operations[operation], kernel, iterations=iterations)

    return improved_mask

In the post-processing stage of my image segmentation pipeline, I have developed the `improve_mask` function. This function is crucial for refining the segmentation masks using morphological operations, enhancing the quality and accuracy of the final output.

**Function: `improve_mask`**
The function's primary objective is to apply specific morphological operations to a binary mask to correct small imperfections and enhance the overall structure of the segmented regions.

- It starts by creating a kernel of the specified size (`kernel_size`), which is used in the morphological operations. The kernel size determines the extent of the operation's effect on the mask.
- A dictionary maps the supported operations (opening, closing, erosion, dilation) to their corresponding OpenCV morphological operation codes. 
- The function then checks if the specified operation is supported and raises a ValueError if it's not.
- Depending on the chosen operation, the function applies it to the provided mask. These operations include:
  - Opening: Useful for removing small objects or noise from the foreground.
  - Closing: Helps to close small holes or gaps within the foreground objects.
  - Erosion: Erodes the boundaries of the foreground, which can be useful for separating closely connected objects.
  - Dilation: Expands the foreground objects, which can help in closing small holes and connecting fragmented components.
- The number of iterations dictates how many times the operation is applied, allowing for more aggressive or subtle modifications.

This function is an integral part of my workflow, especially when dealing with binary masks resulting from segmentation processes. It provides the flexibility to fine-tune the results, ensuring that the final segmented regions align closely with the actual areas of interest in the image.


### Exploring the best operation for the images

In [62]:
def morphology(mask, operations, kernel_sizes):
    num_operations = len(operations)
    num_kernels = len(kernel_sizes)
    total_plots = num_operations * num_kernels  # Total number of plots
    assert total_plots <= 16, "Total number of subplots should be less than or equal to 16"

    results = {op: [] for op in operations}
    plt.figure(figsize=(10, 10))

    # Create a subplot for each combination of operation and kernel size
    subplot_idx = 1  # Initialize subplot index
    for i, operation in enumerate(operations):
        for size in kernel_sizes:
            mask_imp = improve_mask(mask, operation=operation, kernel_size=size)

            ax = plt.subplot(4, 4, subplot_idx)  # Update to use subplot_idx
            ax.imshow(overlay_mask_on_image(thisMask, mask_imp), cmap='gray')
            ax.set_title(f"{operation}, kernel size: {size}")
            ax.axis('off')
            results[f'{operation}'].append(dice_coefficient(mask_imp, thisMask))
            subplot_idx += 1  # Move to the next subplot index

    plt.tight_layout()
    plt.show()
    return results

morf_dice = morphology(region_mask, ['open', 'close', 'erode', 'dilate'], [3, 5, 7, 9])
morf_df = pd.DataFrame(data=morf_dice, index=[3, 5, 7, 9])
morf_df

<IPython.core.display.Javascript object>

,open,close,erode,dilate
3,0.560035,0.676961,0.375188,0.847742
5,0.462121,0.768242,0.207506,0.926145
7,0.427119,0.795866,0.086034,0.914368
9,0.276923,0.848997,0.006139,0.869121


In my exploration of optimizing segmentation results, I've developed a function named `morphology`. This function systematically evaluates different morphological operations and kernel sizes on a segmented mask to determine the most effective combination.

**Function: `morphology`**
The primary goal of this function is to visually and quantitatively assess the impact of various morphological operations and kernel sizes on a given binary mask. It's designed to help identify the best operation and kernel size for refining segmentation masks.

- The function first sets up a structure to store the results and creates a figure for plotting.
- For each combination of morphological operation and kernel size, it calls the `improve_mask` function to apply the operation to the mask.
- Each processed mask is then displayed in a subplot with an appropriate title indicating the operation and kernel size.
- Alongside visual evaluation, I use the `dice_coefficient` function to quantitatively assess the effectiveness of each operation. This metric compares the improved mask against a reference mask (`thisMask`), providing a score that reflects the quality of the segmentation.
- The results are stored in a dictionary, with each operation as a key and a list of Dice coefficients corresponding to different kernel sizes.
- The function limits the total number of subplots to 16 for practical visualization purposes.
- Finally, the results are displayed using `plt.show()`, and the function returns the dictionary of results.

This function is an essential part of my segmentation analysis. It allows me to experiment with different post-processing techniques and quantitatively measure their effectiveness, leading to an informed choice of the best morphological operation and kernel size for enhancing my segmentation masks.


## Close or Dilate?

In [63]:
close_vals = np.arange(1, 100, 2).astype(np.int8)
dice_coefs = {}

for val in close_vals:
    this_mask = improve_mask(region_mask, 
                             operation="dilate",
                             kernel_size=val)
    dice_coefs[val] = dice_coefficient(this_mask, thisMask)

plt.figure(figsize=(9,5))
plt.plot(dice_coefs.keys(), dice_coefs.values(), label="Dice Coefficient Variation")
plt.scatter(x=dice_coefs.keys(), y=dice_coefs.values(), marker="o", c='red')
max_key = max(dice_coefs, key=lambda k: dice_coefs[k].round(3))
plt.annotate(text=f"Kernel size: {max_key}, value: {dice_coefs[max_key]:.3f}", xy=(max_key, dice_coefs[max_key]),
             xytext=(max_key-4.5, dice_coefs[max_key]+.01))
plt.axvline(x=max_key, ymax=dice_coefs[max_key]/1.01, color='green', linestyle='--')
plt.axhline(y=dice_coefs[max_key], xmax=max_key/90, color='green', linestyle='--')
plt.xlabel("Kernel Size")
plt.ylabel("Dice Coefficient")
plt.yticks(np.linspace(.1, 1, num=10).round(2))
plt.title("Dice Coefficient vs. Kernel Size for Morphological Closing")
plt.legend()
plt.show()

<IPython.core.display.Javascript object>

In [64]:
close_vals = np.arange(1, 100, 2).astype(np.int8)
dice_coefs = {}

for val in close_vals:
    this_mask = improve_mask(region_mask,
                             operation="close",
                             kernel_size=val)
    dice_coefs[val] = dice_coefficient(this_mask, thisMask)

plt.figure(figsize=(9,5))
plt.plot(dice_coefs.keys(), dice_coefs.values(), label="Dice Coefficient Variation")
plt.scatter(x=dice_coefs.keys(), y=dice_coefs.values(), marker="o", c='red')
max_key = max(dice_coefs, key=lambda k: dice_coefs[k].round(3))
plt.annotate(text=f"Kernel size: {max_key}, value: {dice_coefs[max_key]:.3f}", xy=(max_key, dice_coefs[max_key]),
             xytext=(max_key-4.5, dice_coefs[max_key]+.01))
plt.axvline(x=max_key, ymax=dice_coefs[max_key]/1.01, color='green', linestyle='--')
plt.axhline(y=dice_coefs[max_key], xmax=max_key/90, color='green', linestyle='--')
plt.xlabel("Kernel Size")
plt.ylabel("Dice Coefficient")
plt.yticks(np.linspace(.1, 1, num=10).round(2))
plt.title("Dice Coefficient vs. Kernel Size for Morphological Closing")
plt.legend()
plt.show()

<IPython.core.display.Javascript object>

### Mask evolution in postprocessing

In [65]:
this_mask = improve_mask(region_mask, operation="dilate", kernel_size=5)
visualize(image=thisMRI, 
          truth=thisMask,
          prediction=this_mask)
print(f'Dilate: {dice_coefficient(this_mask, thisMask)}')

this_mask = improve_mask(this_mask, operation='close', kernel_size=5) # 0.87979 -- 9
visualize(image=thisMRI,
          truth=thisMask,
          prediction=this_mask)
print(f'close: {dice_coefficient(this_mask, thisMask)}')

visualize(image=thisMRI,
          truth=thisMask,
          prediction=this_mask)

<IPython.core.display.Javascript object>

Dilate: 0.9261452165783733


<IPython.core.display.Javascript object>

close: 0.9421133231240428


<IPython.core.display.Javascript object>

## Experiments

In [66]:
def complete_workflow(input_image, mask, fuzzy_th, num_seeds, sigma, dilation_kernel_size):
    """
    Executes a full workflow of image processing for segmenting MRI images, which includes:
    feature extraction, automatic seed placement, Gaussian filtering, and mask improvement.

    Parameters:
    - im: The input MRI image to process.
    - mask: The gold standard mask for the MRI image.
    - fuzzy_th: The fuzzy threshold value to use for segmentation.
    - num_seeds: The number of seeds to distribute for region growing.

    Returns:
    - A mask after performing fuzzy thresholding, seeding, and morphological operations.
    """
    
    # Automatic seed distribution
    sd, _ = automatic_seed_distribution(input_image, mask, num_seeds, 20, 2)

    # MRI preprocessing with Gaussian filtering
    tfinalmask = cv2.GaussianBlur(input_image, (0,0), sigma)

    # Fuzzy thresholding and seeding
    sd = [(y, x) for x, y in sd]  # Reformat seed positions to (y, x) for the growing algorithm
    rmask = region_grow_all_seeds(tfinalmask, sd, fuzzy_th)

    # Morphological improvements to the grown regions
    mask_dilated = improve_mask(rmask, operation="dilate", kernel_size=dilation_kernel_size)
    final_mask = improve_mask(mask_dilated, operation='close', kernel_size=5)

    return final_mask


In my project on MRI image segmentation, I have developed a comprehensive function called `complete_workflow`. This function encapsulates the entire process of segmenting MRI images, from initial seed placement to final mask refinement.

**Function: `complete_workflow`**
This function integrates multiple steps of the segmentation process, ensuring a seamless transition from one stage to another to produce an optimal segmentation mask.

- **Automatic Seed Distribution**: The function begins with the distribution of seeds across the image. This step is crucial for initializing the region-growing process. The `automatic_seed_distribution` function places a specified number of seeds (`num_seeds`) based on the characteristics of the input image and mask.
  
- **MRI Preprocessing with Gaussian Filtering**: To enhance the features of the MRI image and reduce noise, the function applies Gaussian filtering. This step smooths the image, making it more amenable to accurate segmentation. The standard deviation of the Gaussian kernel is controlled by the `sigma` parameter.

- **Fuzzy Thresholding and Seeding**: Following preprocessing, the function performs fuzzy thresholding and seeding using the `region_grow_all_seeds` function. It grows regions from the distributed seeds based on the specified fuzzy threshold (`fuzzy_th`), leading to the initial segmentation mask.

- **Morphological Improvements**: The final step involves refining the grown regions using morphological operations. Initially, the mask is dilated to fill gaps and connect disjointed regions, controlled by `dilation_kernel_size`. Subsequently, a closing operation (dilation followed by erosion) is applied to smooth the mask edges and improve the overall shape.


### Experiment 1

In [ ]:
path2 = "C://Users\mario\Python\Projects\MRIfuzzySegmentation\Sample data"
mri_image = nib.load(os.path.join(path2, "sub-r001s001_ses-1_space-MNI152NLin2009aSym_T1w.nii.gz")).get_fdata()
mri_mask = nib.load(os.path.join(path2, "sub-r001s001_ses-1_space-MNI152NLin2009aSym_label-L_desc-T1lesion_mask.nii.gz")).get_fdata()
# Prepare a dictionary to store the best fuzzy value, dice score, and number of seeds for each slice
best_fuzzy_per_slice = {}

# Iterate over each slice
for val in np.arange(50, 138):  # Assuming these are your slice indices
    best_dice_score = -1  # Initialize with a negative score
    best_parameters = {'fuzzy_val': None, 'num_seeds': None}  # Placeholder for the best parameters
    best_mask = None  # Placeholder for the best mask corresponding to the best parameters
    
    # Iterate over a range of number of seeds to find the best one for this slice
    for num_seeds in range(4, 12):  # Adjust the range as needed
        for fuzzy_val in np.linspace(0.01, 1, 30):  # Fuzzy values to try
            try:
                num_slice = val
                mri_image_slice = mri_image[:, :, val].astype(np.uint8)
                mri_mask_slice = mri_mask[:, :, val].astype(np.uint8) * 255
                # Run the complete workflow with the current number of seeds and fuzzy value
                this_final = complete_workflow(mri_image_slice, mri_mask_slice, fuzzy_val, num_seeds, 2, 7)
                
                # Calculate the dice score for the current parameters
                current_dice_score = dice_coefficient(this_final, mri_mask_slice)
                
                # If the current dice score is higher than the previous best, update the best score and parameters
                if current_dice_score > best_dice_score:
                    best_dice_score = current_dice_score
                    best_parameters = {'fuzzy_val': fuzzy_val, 'num_seeds': num_seeds}
                    best_mask = this_final.copy()  # Make sure to copy the mask to avoid overwriting it in future iterations
            
            except Exception as e:  # Catch any exceptions that occur during processing
                print(f'Failure in processing slice {val} with {num_seeds} seeds: {e}')
                continue
    
    # Store the best fuzzy value, dice score, and number of seeds for the current slice in the dictionary
    best_fuzzy_per_slice[val] = {
        'parameters': best_parameters,
        'dice_score': best_dice_score,
        'mask': best_mask
    }
    print(f'Slice {val} completed')
    print(f'Data for slice {val}: \n{best_fuzzy_per_slice[val]}')

def process_and_save_data_experiment1(data_dict, csv_file_path, nii_file_path):
    # Preparar listas para almacenar los datos de parámetros y las máscaras
    fuzzy_vals = []
    num_seeds_list = []
    dice_scores = []
    masks = []

    # Iterar sobre los elementos del diccionario para extraer los datos
    for val in data_dict.values():
        fuzzy_vals.append(val['parameters']['fuzzy_val'])
        num_seeds_list.append(val['parameters']['num_seeds'])
        dice_scores.append(val['dice_score'])
        masks.append(val['mask'])

    # Guardar los parámetros en un DataFrame y luego a un archivo CSV
    param_df = pd.DataFrame({
        'fuzzy_val': fuzzy_vals,
        'num_seeds': num_seeds_list,
        'dice_score': dice_scores
    })
    param_df.to_csv(csv_file_path, index=False)
    print(f"Parámetros guardados en {csv_file_path}")

    # Apilar todas las máscaras y guardarlas en un único archivo .nii.gz
    stacked_masks = np.stack(masks, axis=-1)
    nifti_mask_stack = nib.Nifti1Image(stacked_masks.astype(np.uint8), np.eye(4))
    nib.save(nifti_mask_stack, nii_file_path)
    print(f"Máscaras guardadas en {nii_file_path}")

path = "C://Users\mario\Python\Projects\MRIfuzzySegmentation\OUTPUT_files\experiment1_data"
process_and_save_data_experiment1(
    data_dict=best_fuzzy_per_slice,  # El diccionario que tienes
    csv_file_path= os.path.join(path, 'best_params_exp1.csv'),  # La ruta donde quieres guardar el CSV
    nii_file_path=os.path.join(path, 'all_masks_exp1.nii.gz')  # La ruta donde quieres guardar las máscaras
)

1. **Iterative Segmentation Analysis**: 
   - For each slice of the MRI image, I systematically test a range of fuzzy threshold values and seed counts to find the combination that yields the best segmentation result, as measured by the Dice coefficient. 
   - This process utilizes a custom workflow function, `complete_workflow`, which performs segmentation based on the given parameters for each slice.

2. **Data Aggregation and Analysis**: 
   - Upon finding the best parameters for each slice, I store this data in a dictionary, `best_fuzzy_per_slice`. This dictionary maps each slice to its optimal segmentation parameters and the corresponding Dice score.

3. **Data Processing and Saving**: 
   - With the `process_and_save_data_experiment1` function, I extract and compile the optimal parameters and the Dice scores from the dictionary into a pandas DataFrame. This DataFrame is then saved as a CSV file for easy access and analysis.
   - Additionally, I stack the segmentation masks generated for each slice and save them as a single `.nii.gz` file. This consolidated file provides a comprehensive view of the segmentation results across all slices.

4. **Execution and Output Generation**: 
   - The entire process is executed for the specified range of slices of the MRI image, and the results are saved in a designated output directory.


### Experiment 2

In [ ]:
path2 = "C://Users\mario\Python\Projects\MRIfuzzySegmentation\Sample data"
mri_image = nib.load(os.path.join(path2, "sub-r001s001_ses-1_space-MNI152NLin2009aSym_T1w.nii.gz")).get_fdata()
mri_mask = nib.load(os.path.join(path2, "sub-r001s001_ses-1_space-MNI152NLin2009aSym_label-L_desc-T1lesion_mask.nii.gz")).get_fdata()

best_fuzzy_per_slice = {}

for val in np.arange(50, 138):  # Assuming these are your slice indices
    best_dice_score = -1
    best_parameters = {'fuzzy_val': None, 'num_seeds': None, 'sigma': None}
    best_mask = None

    for num_seeds in range(4, 12):  # Adjust the range as needed
        for fuzzy_val in np.linspace(0.01, 1, 30):  # Fuzzy values to try
            for sigma in np.linspace(1, 5, 5):  # Example range for sigma
                try:
                    mri_image_slice = mri_image[:, :, val].astype(np.uint8)
                    mri_mask_slice = mri_mask[:, :, val].astype(np.uint8) * 255
                    this_final = complete_workflow(mri_image_slice, mri_mask_slice, fuzzy_val, num_seeds, sigma, 7)

                    current_dice_score = dice_coefficient(this_final, mri_mask_slice)

                    if current_dice_score > best_dice_score:
                        best_dice_score = current_dice_score
                        best_parameters = {'fuzzy_val': fuzzy_val, 'num_seeds': num_seeds, 'sigma': sigma}
                        best_mask = this_final.copy()

                except Exception as e:
                    print(f'Failure in processing slice {val} with {num_seeds} seeds and sigma {sigma}: {e}')
                    continue

    best_fuzzy_per_slice[val] = {
        'parameters': best_parameters,
        'dice_score': best_dice_score,
        'mask': best_mask
    }
    print(f'Slice {val} completed')
    print(f'Data for slice {val}: \n{best_fuzzy_per_slice[val]}')

def process_and_save_data_experiment1(data_dict, csv_file_path, nii_file_path):
    fuzzy_vals = []
    num_seeds_list = []
    sigma_list = []
    dice_scores = []
    masks = []

    for val in data_dict.values():
        fuzzy_vals.append(val['parameters']['fuzzy_val'])
        num_seeds_list.append(val['parameters']['num_seeds'])
        sigma_list.append(val['parameters']['sigma'])
        dice_scores.append(val['dice_score'])
        masks.append(val['mask'])

    param_df = pd.DataFrame({
        'fuzzy_val': fuzzy_vals,
        'num_seeds': num_seeds_list,
        'sigma': sigma_list,
        'dice_score': dice_scores
    })
    param_df.to_csv(csv_file_path, index=False)
    print(f"Parámetros guardados en {csv_file_path}")

    stacked_masks = np.stack(masks, axis=-1)
    nifti_mask_stack = nib.Nifti1Image(stacked_masks.astype(np.uint8), np.eye(4))
    nib.save(nifti_mask_stack, nii_file_path)
    print(f"Máscaras guardadas en {nii_file_path}")

path = "C://Users\mario\Python\Projects\MRIfuzzySegmentation\OUTPUT_files\experiment2_data"
process_and_save_data_experiment1(
    data_dict=best_fuzzy_per_slice,
    csv_file_path=os.path.join(path, 'best_params_exp2.csv'),
    nii_file_path=os.path.join(path, 'all_masks_exp2.nii.gz')
)


1. **Data Loading and Preparation**:
   - MRI image data and its corresponding mask are loaded from a specified directory using the `nibabel` library, which handles neuroimaging data formats like `.nii.gz`.
   - A dictionary, `best_fuzzy_per_slice`, is prepared to store the best segmentation parameters and results for each slice of the MRI image.

2. **Segmentation Parameter Optimization**:
   - For each slice in the specified range (here, slices 50 to 138), I perform an exhaustive search over a range of seed numbers, fuzzy values, and sigma values for Gaussian blurring.
   - The `complete_workflow` function is used for each combination of parameters to segment the slice. This function integrates seed distribution, Gaussian filtering, and segmentation.
   - The effectiveness of each parameter combination is evaluated using the Dice coefficient, a statistical tool to measure the similarity between the segmented mask and the gold standard mask.
   - The best performing parameters and the corresponding mask for each slice are stored in the `best_fuzzy_per_slice` dictionary.

3. **Data Aggregation and Storage**:
   - The `process_and_save_data_experiment1` function processes the collected data. It extracts the optimal segmentation parameters and Dice scores from the dictionary for all slices and compiles this information into a pandas DataFrame.
   - This DataFrame, containing details of the best fuzzy values, seed numbers, sigma values, and Dice scores, is saved as a CSV file for easy access and analysis.
   - Additionally, the segmented masks for all slices are stacked and saved as a single `.nii.gz` file. This file provides a comprehensive view of the segmentation results across the slices.

4. **Execution and Output**:
   - The entire process is executed, and the results, including the parameter DataFrame and the stacked masks, are saved in a designated output directory (`experiment2_data`).


### Experiment 3

In [24]:
def process_and_save_data(data_dict, csv_file_path, nii_file_path):
    # Preparar listas para almacenar los datos de parámetros y las máscaras
    fuzzy_vals = []
    num_seeds_list = []
    sigmas = []
    dice_scores = []
    masks = []
    dilation_kernels_sizes = []

    # Iterar sobre los elementos del diccionario para extraer los datos
    for val in data_dict.values():
        fuzzy_vals.append(val['parameters']['fuzzy_val'])
        num_seeds_list.append(val['parameters']['num_seeds'])
        sigmas.append(val['parameters']['sigma'])
        dilation_kernels_sizes.append(val['parameters']['dilation_kernel'])
        dice_scores.append(val['dice_score'])
        masks.append(val['mask'])

    # Guardar los parámetros en un DataFrame y luego a un archivo CSV
    param_df = pd.DataFrame({
        'fuzzy_val': fuzzy_vals,
        'num_seeds': num_seeds_list,
        'sigma': sigmas,
        'dilation_kernel': dilation_kernels_sizes,
        'dice_score': dice_scores
    })
    param_df.to_csv(csv_file_path, index=False)
    print(f"Parámetros guardados en {csv_file_path}")

    # Apilar todas las máscaras y guardarlas en un único archivo .nii.gz
    stacked_masks = np.stack(masks, axis=-1)
    nifti_mask_stack = nib.Nifti1Image(stacked_masks.astype(np.uint8), np.eye(4))
    nib.save(nifti_mask_stack, nii_file_path)
    print(f"Máscaras guardadas en {nii_file_path}")

In [ ]:
# Prepare a dictionary to store the best parameters and mask for each slice
best_parameters_per_slice = {}
# Iterate over each slice
for val in np.arange(50, 138):  # Assuming these are your slice indices
    best_dice_score = -1  # Initialize with a negative score
    best_parameters = {'fuzzy_val': None, 'num_seeds': None, 'sigma': None, 'dilation_kernel': None}  # Placeholder for the best parameters
    best_mask = None  # Placeholder for the best mask corresponding to the best parameters

    # Iterate over a range of number of seeds to find the best one for this slice
    for dilation_kernel_size in [3, 5, 7]:  
        for num_seeds in range(2, 12):  # Adjust the range as needed
            for fuzzy_val in np.linspace(0.01, 1, 30):  # Fuzzy values to try
                for sigma in np.linspace(2, 5, 7):  # Sigma values for Gaussian blur to try
                    try:
                        num_slice = val
                        v2mri_image = mri_image[:, :, num_slice].astype(np.uint8)
                        v2mri_mask = mri_mask[:, :, num_slice].astype(np.uint8) * 255
                        
                        # Run the complete workflow with the current parameters
                        this_final_mask = complete_workflow(v2mri_image, v2mri_mask, fuzzy_val, num_seeds, sigma, dilation_kernel_size)
                        
                        # Calculate the dice score for the current mask
                        current_dice_score = dice_coefficient(this_final_mask, v2mri_mask)
    
                        # If the current dice score is higher than the previous best, update the best score and parameters
                        if current_dice_score > best_dice_score:
                            best_dice_score = current_dice_score
                            best_parameters = {'fuzzy_val': fuzzy_val, 'num_seeds': num_seeds, 'sigma': sigma, 'dilation_kernel': dilation_kernel_size}
                            best_mask = this_final_mask.copy()  # Make sure to copy the mask to avoid overwriting it in future iterations
                        
                    except Exception as e:  # Catch any exceptions that occur during processing
                        print(f'Failure in processing slice {val} with seeds: {num_seeds}, sigma: {sigma}: {e}')
        
    # Store the best parameters and dice score for the current slice in the dictionary
    best_parameters_per_slice[val] = {
        'parameters': best_parameters,
        'dice_score': best_dice_score,
        'mask': best_mask
    }
    print(f'Slice {val} completed')
    print(f'Data for slice {val}: \n{best_parameters_per_slice[val]}')


path = "/home/mariopasc/Python/Projects/MRIfuzzySegmentation/OUTPUT_files/best_params_dilation"
process_and_save_data(
    data_dict=best_parameters_per_slice,  # El diccionario que tienes
    csv_file_path= os.path.join(path, 'best_parameters_dilat.csv'),  # La ruta donde quieres guardar el CSV
    nii_file_path=os.path.join(path, 'all_masks.nii.gz')  # La ruta donde quieres guardar las máscaras
)

1. **Initialization and Iterative Analysis**:
   - A dictionary, `best_parameters_per_slice`, is prepared to store the optimal parameters and the resulting mask for each slice.
   - For each slice in the MRI image, represented by indices from 50 to 138, I conduct a comprehensive search across different ranges of dilation kernel sizes, seed numbers, fuzzy values, and sigma values.
   
2. **Segmentation and Parameter Optimization**:
   - The `complete_workflow` function is employed for each combination of parameters to segment the MRI slice. This integrated function performs the entire segmentation process, including automatic seed placement, Gaussian filtering, and morphological improvements.
   - The effectiveness of each parameter set is evaluated using the Dice coefficient, which measures the similarity between the segmented mask and the standard mask.
   - The best-performing parameters and their corresponding masks for each slice are stored in the `best_parameters_per_slice` dictionary.

3. **Data Aggregation and Saving**:
   - The `process_and_save_data` function is utilized to organize and store the collected data. It compiles the optimal segmentation parameters and Dice scores into a pandas DataFrame and saves this data as a CSV file for further analysis.
   - Additionally, the segmented masks for each slice are stacked and saved in a single `.nii.gz` file, providing a comprehensive view of the segmentation results.

4. **Execution and Output**:
   - The process is executed for the specified range of slices, and the results, including the DataFrame of parameters and the stacked segmentation masks, are saved in a designated directory (`/home/mariopasc/Python/Projects/MRIfuzzySegmentation/OUTPUT_files/best_params_dilation`).
